In [15]:
# !pip install -r ./requirements.txt

In [6]:
# !pip uninstall virny -y

In [8]:
# Install using an HTTP link
# !pip install git+https://github.com/DataResponsibly/Virny.git@development

# Install using an SSH link
# !pip install git+ssh://git@github.com/DataResponsibly/Virny.git@development

In [9]:
# !pip install aif360

In [10]:
# !pip install BlackBoxAuditing==0.1.54

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [3]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "fairness-variance":
    os.chdir("../..")

print('Current location: ', os.getcwd())

Current location:  /home/dh3553/projects/fairness-variance


## Import dependencies

In [4]:
import os
import copy

from virny.utils.custom_initializers import create_config_obj
from virny.datasets import LawSchoolDataset

from configs.constants import TEST_SET_FRACTION, EXPERIMENT_SEEDS
from configs.models_config_for_tuning import get_model_params_for_mult_repair_levels

from source.preprocessing import get_simple_preprocessor
from source.experiment_interface import run_exp_iter_with_disparate_impact

pip install 'aif360[LawSchoolGPA]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'


## Define Input Variables

In [5]:
# ROOT_DIR = os.path.join(os.getcwd(), "..", "..")
ROOT_DIR = os.getcwd()
EXPERIMENT_NAME = 'mult_repair_levels_law_school'
DB_COLLECTION_NAME = 'exp_mult_repair_levels'
DATASET_NAME = 'LawSchoolDataset'
SAVE_RESULTS_DIR_PATH = os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME)
# FAIR_INTERVENTION_PARAMS_LST = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
FAIR_INTERVENTION_PARAMS_LST = [0.0, 0.2, 0.3, 0.5]

config_yaml_path = os.path.join(ROOT_DIR, 'notebooks', EXPERIMENT_NAME, 'law_school_2018_config.yaml')
metrics_computation_config = create_config_obj(config_yaml_path=config_yaml_path)

## Define a db writer and custom fields to insert into your database

In [6]:
import os
from dotenv import load_dotenv

load_dotenv('./configs/secrets.env')
os.getenv("DB_NAME")

'fairness_variance'

In [7]:
from source.utils.db_functions import connect_to_mongodb

client, collection_obj, db_writer_func = connect_to_mongodb(DB_COLLECTION_NAME)

In [8]:
import uuid

custom_table_fields_dct = {
#     'session_uuid': str(uuid.uuid4()),
    'session_uuid': '9430484b-bec6-42a0-ac65-11c53446a61f',
}
print('Current session uuid: ', custom_table_fields_dct['session_uuid'])

Current session uuid:  9430484b-bec6-42a0-ac65-11c53446a61f


## Initialize custom objects

In [9]:
data_loader = LawSchoolDataset()
data_loader.X_data.head()

,decile1b,decile3,lsat,ugpa,zfygpa,zgpa,fulltime,fam_inc,male,tier,race
0,10.0,10.0,44.0,3.5,1.33,1.88,1.0,5.0,0.0,4.0,White
1,5.0,4.0,29.0,3.5,-0.11,-0.57,1.0,4.0,0.0,2.0,White
2,8.0,7.0,37.0,3.4,0.63,0.37,1.0,3.0,1.0,4.0,White
3,8.0,7.0,43.0,3.3,0.67,0.34,1.0,4.0,0.0,4.0,White
4,3.0,2.0,41.0,3.3,-0.67,-1.30,1.0,4.0,0.0,5.0,White


In [10]:
data_loader.X_data.shape

(20798, 11)

In [11]:
data_loader.X_data.dtypes

decile1b    float64
decile3     float64
lsat        float64
ugpa        float64
zfygpa      float64
zgpa        float64
fulltime     object
fam_inc      object
male         object
tier         object
race         object
dtype: object

In [12]:
data_loader.y_data.value_counts()

1.0    18505
0.0     2293
Name: pass_bar, dtype: int64

In [13]:
data_loader.X_data['race'].value_counts()

White        17491
Non-White     3307
Name: race, dtype: int64

In [14]:
data_loader.X_data['male'].value_counts()

1.0    11675
0.0     9123
Name: male, dtype: int64

## Run experiment iterations

### Experiment iteration 1

In [79]:
# tuned_params_filenames = ['tuning_results_Folktables_NY_2018_Employment_alpha_0.8_20230706__115508.csv']
# tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
#                          for tuned_params_filename in tuned_params_filenames]

In [80]:
# Configs for an experiment iteration
exp_iter_num = 1
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_model_params_for_mult_repair_levels(experiment_seed)

In [81]:
run_exp_iter_with_disparate_impact(data_loader=exp_iter_data_loader,
                                   experiment_seed=experiment_seed,
                                   test_set_fraction=TEST_SET_FRACTION,
                                   db_writer_func=db_writer_func,
                                   fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                   models_params_for_tuning=models_params_for_tuning,
                                   metrics_computation_config=metrics_computation_config,
                                   custom_table_fields_dct=custom_table_fields_dct,
                                   with_tuning=True,
                                   # with_tuning=False,
                                   # tuned_params_df_paths=tuned_params_df_paths,
                                   save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                   verbose=True, 
                                   dataset_name=DATASET_NAME)

2023-08-06 15:52:20 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 100,
 'experiment_iteration': 'Exp_iter_1',
 'fair_intervention_params_lst': '[0.1, 0.3]',
 'model_init_seed': 100,
 'session_uuid': '9430484b-bec6-42a0-ac65-11c53446a61f'}




2023-08-06 15:52:20 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed


Top indexes of an X_test in a base flow dataset:  Int64Index([ 7102,   593, 18841,  5078, 14172,  8064, 13554, 13401, 17015,
            18446,  6938,  3450,  9375, 19994, 16100,  4401,   142, 15143,
             2188,  4332],
           dtype='int64')
Top indexes of an y_test in a base flow dataset:  Int64Index([ 7102,   593, 18841,  5078, 14172,  8064, 13554, 13401, 17015,
            18446,  6938,  3450,  9375, 19994, 16100,  4401,   142, 15143,
             2188,  4332],
           dtype='int64')


Multiple alphas:   0%|          | 0/2 [00:00<?, ?it/s]

...

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

intervention_param:  0.3
2023/08/06, 16:24:58: Tuning RandomForestClassifier...
2023/08/06, 16:31:10: Tuning for RandomForestClassifier is finished [F1 score = 0.6450669297827822, Accuracy = 0.8963141025641025]



Multiple runs progress:   0%|          | 0/1 [00:00<?, ?it/s]

Analyze models in one run:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 2

In [15]:
# Configs for an experiment iteration
exp_iter_num = 2
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Law_School_alpha_0.0_20230806__154602.csv',
    'tuning_results_Law_School_alpha_0.2_20230806__161805.csv',
    'tuning_results_Law_School_alpha_0.3_20230806__203110.csv',
    'tuning_results_Law_School_alpha_0.5_20230806__195848.csv',
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_model_params_for_mult_repair_levels(experiment_seed)

In [ ]:
run_exp_iter_with_disparate_impact(data_loader=exp_iter_data_loader,
                                   experiment_seed=experiment_seed,
                                   test_set_fraction=TEST_SET_FRACTION,
                                   db_writer_func=db_writer_func,
                                   fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                   models_params_for_tuning=models_params_for_tuning,
                                   metrics_computation_config=metrics_computation_config,
                                   custom_table_fields_dct=custom_table_fields_dct,
#                                    with_tuning=True,
                                   with_tuning=False,
                                   tuned_params_df_paths=tuned_params_df_paths,
                                   save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                   verbose=True, 
                                   dataset_name=DATASET_NAME)

2023-08-06 17:50:11 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 200,
 'experiment_iteration': 'Exp_iter_2',
 'fair_intervention_params_lst': '[0.1, 0.9]',
 'model_init_seed': 200,
 'session_uuid': '9430484b-bec6-42a0-ac65-11c53446a61f'}




2023-08-06 17:50:11 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed


Top indexes of an X_test in a base flow dataset:  Int64Index([11154,  2918,  4165, 16989, 11116, 18464,  5262,  2614, 11058,
            14769, 11087, 10125, 19318, 10215, 10253,  7587,  2716,   577,
             2257,  1011],
           dtype='int64')
Top indexes of an y_test in a base flow dataset:  Int64Index([11154,  2918,  4165, 16989, 11116, 18464,  5262,  2614, 11058,
            14769, 11087, 10125, 19318, 10215, 10253,  7587,  2716,   577,
             2257,  1011],
           dtype='int64')


Multiple alphas:   0%|          | 0/2 [00:00<?, ?it/s]

...

Multiple runs progress:   0%|          | 0/1 [00:00<?, ?it/s]

Analyze models in one run:   0%|          | 0/1 [00:00<?, ?it/s]

##############################  [Model 1 / 1] Analyze RandomForestClassifier  ##############################
Model seed:  201

Protected groups splits:
male_priv (1191, 2)
male_dis (889, 2)
race_priv (1765, 2)
race_dis (315, 2)
male&race_priv (1927, 2)
male&race_dis (153, 2)




2023-08-06 17:50:13 abstract_overall_variance_analyzer.py INFO    : Start classifiers testing by bootstrap
INFO:root:Start classifiers testing by bootstrap


Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 3

In [15]:
# Configs for an experiment iteration
exp_iter_num = 3
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Law_School_alpha_0.0_20230806__154602.csv',
    'tuning_results_Law_School_alpha_0.2_20230806__161805.csv',
    'tuning_results_Law_School_alpha_0.3_20230806__203110.csv',
    'tuning_results_Law_School_alpha_0.5_20230806__195848.csv',
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_model_params_for_mult_repair_levels(experiment_seed)

In [ ]:
run_exp_iter_with_disparate_impact(data_loader=exp_iter_data_loader,
                                   experiment_seed=experiment_seed,
                                   test_set_fraction=TEST_SET_FRACTION,
                                   db_writer_func=db_writer_func,
                                   fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                   models_params_for_tuning=models_params_for_tuning,
                                   metrics_computation_config=metrics_computation_config,
                                   custom_table_fields_dct=custom_table_fields_dct,
#                                    with_tuning=True,
                                   with_tuning=False,
                                   tuned_params_df_paths=tuned_params_df_paths,
                                   save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                   verbose=True, 
                                   dataset_name=DATASET_NAME)

2023-08-06 17:58:51 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 300,
 'experiment_iteration': 'Exp_iter_3',
 'fair_intervention_params_lst': '[0.4, 0.6, 0.7, 0.8, 1.0]',
 'model_init_seed': 300,
 'session_uuid': '9430484b-bec6-42a0-ac65-11c53446a61f'}




2023-08-06 17:58:51 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed


Top indexes of an X_test in a base flow dataset:  Int64Index([19898, 14980, 13028,  5574, 17599,  3210, 11359, 19768, 19647,
             5966, 12835,  9832,  4098,  6607, 11751,   861, 11502, 17873,
            10882, 17470],
           dtype='int64')
Top indexes of an y_test in a base flow dataset:  Int64Index([19898, 14980, 13028,  5574, 17599,  3210, 11359, 19768, 19647,
             5966, 12835,  9832,  4098,  6607, 11751,   861, 11502, 17873,
            10882, 17470],
           dtype='int64')


Multiple alphas:   0%|          | 0/5 [00:00<?, ?it/s]

...

Multiple runs progress:   0%|          | 0/1 [00:00<?, ?it/s]

Analyze models in one run:   0%|          | 0/1 [00:00<?, ?it/s]

##############################  [Model 1 / 1] Analyze RandomForestClassifier  ##############################
Model seed:  301

Protected groups splits:
male_priv (1165, 2)
male_dis (915, 2)
race_priv (1760, 2)
race_dis (320, 2)
male&race_priv (1908, 2)
male&race_dis (172, 2)




2023-08-06 17:58:52 abstract_overall_variance_analyzer.py INFO    : Start classifiers testing by bootstrap
INFO:root:Start classifiers testing by bootstrap


Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 4

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 4
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Law_School_alpha_0.0_20230806__154602.csv',
    'tuning_results_Law_School_alpha_0.2_20230806__161805.csv',
    'tuning_results_Law_School_alpha_0.3_20230806__203110.csv',
    'tuning_results_Law_School_alpha_0.5_20230806__195848.csv',
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_model_params_for_mult_repair_levels(experiment_seed)

In [46]:
run_exp_iter_with_disparate_impact(data_loader=exp_iter_data_loader,
                                   experiment_seed=experiment_seed,
                                   test_set_fraction=TEST_SET_FRACTION,
                                   db_writer_func=db_writer_func,
                                   fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                   models_params_for_tuning=models_params_for_tuning,
                                   metrics_computation_config=metrics_computation_config,
                                   custom_table_fields_dct=custom_table_fields_dct,
#                                    with_tuning=True,
                                   with_tuning=False,
                                   tuned_params_df_paths=tuned_params_df_paths,
                                   save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                   verbose=True, 
                                   dataset_name=DATASET_NAME)

### Experiment iteration 5

In [15]:
# Configs for an experiment iteration
exp_iter_num = 5
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Law_School_alpha_0.0_20230806__154602.csv',
    'tuning_results_Law_School_alpha_0.2_20230806__161805.csv',
    'tuning_results_Law_School_alpha_0.3_20230806__203110.csv',
    'tuning_results_Law_School_alpha_0.5_20230806__195848.csv',
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_model_params_for_mult_repair_levels(experiment_seed)

In [16]:
run_exp_iter_with_disparate_impact(data_loader=exp_iter_data_loader,
                                   experiment_seed=experiment_seed,
                                   test_set_fraction=TEST_SET_FRACTION,
                                   db_writer_func=db_writer_func,
                                   fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                   models_params_for_tuning=models_params_for_tuning,
                                   metrics_computation_config=metrics_computation_config,
                                   custom_table_fields_dct=custom_table_fields_dct,
#                                    with_tuning=True,
                                   with_tuning=False,
                                   tuned_params_df_paths=tuned_params_df_paths,
                                   save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                   verbose=True, 
                                   dataset_name=DATASET_NAME)

2023-08-06 18:10:52 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 500,
 'experiment_iteration': 'Exp_iter_5',
 'fair_intervention_params_lst': '[0.0, 0.2, 0.3, 0.5]',
 'model_init_seed': 500,
 'session_uuid': '9430484b-bec6-42a0-ac65-11c53446a61f'}




2023-08-06 18:10:52 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed


Top indexes of an X_test in a base flow dataset:  Int64Index([12269, 18551, 11422,  4842,  2061, 12046, 16727, 11861, 14580,
            19701, 12917,  7538, 17610, 19120,  5581, 18072, 13209, 20500,
            14741, 11309],
           dtype='int64')
Top indexes of an y_test in a base flow dataset:  Int64Index([12269, 18551, 11422,  4842,  2061, 12046, 16727, 11861, 14580,
            19701, 12917,  7538, 17610, 19120,  5581, 18072, 13209, 20500,
            14741, 11309],
           dtype='int64')


Multiple alphas:   0%|          | 0/4 [00:00<?, ?it/s]

...

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

2023-08-06 19:04:19 abstract_overall_variance_analyzer.py INFO    : Successfully tested classifiers by bootstrap
INFO:root:Successfully tested classifiers by bootstrap
2023-08-06 19:04:36 abstract_overall_variance_analyzer.py INFO    : Successfully computed predict proba metrics
INFO:root:Successfully computed predict proba metrics



[RandomForestClassifier] Metrics matrix:


,Metric,overall,male_priv,male_priv_correct,male_priv_incorrect,male_dis,male_dis_correct,male_dis_incorrect,race_priv,race_priv_correct,...,race_dis_incorrect,male&race_priv,male&race_priv_correct,male&race_priv_incorrect,male&race_dis,male&race_dis_correct,male&race_dis_incorrect,Model_Seed,Model_Name,Model_Params
0,Mean,0.119375,0.123493,0.103941,0.311793,0.113976,0.090360,0.314092,0.093918,0.077907,...,0.369674,0.107978,0.090565,0.295120,0.241909,0.196839,0.378146,501,RandomForestClassifier,"{'bootstrap': False, 'ccp_alpha': 0.0, 'class_..."
1,Std,0.043946,0.045053,0.040862,0.085417,0.042493,0.037159,0.087699,0.039313,0.035554,...,0.091723,0.041832,0.037857,0.084557,0.066670,0.057790,0.093513,501,RandomForestClassifier,"{'bootstrap': False, 'ccp_alpha': 0.0, 'class_..."
2,IQR,0.060974,0.062556,0.056670,0.119234,0.058901,0.051225,0.123943,0.054168,0.048887,...,0.130675,0.057897,0.052290,0.118158,0.094056,0.081053,0.133362,501,RandomForestClassifier,"{'bootstrap': False, 'ccp_alpha': 0.0, 'class_..."
3,Aleatoric_Uncertainty,0.346335,0.355181,0.316743,0.725359,0.334738,0.290353,0.710840,0.302822,0.270432,...,0.782178,0.326420,0.291784,0.698649,0.560452,0.483733,0.792353,501,RandomForestClassifier,"{'bootstrap': False, 'ccp_alpha': 0.0, 'class_..."
4,Overall_Uncertainty,0.366368,0.375584,0.336028,0.756539,0.354284,0.308165,0.745088,0.321586,0.288041,...,0.815148,0.345857,0.310030,0.730881,0.586895,0.507700,0.826281,501,RandomForestClassifier,"{'bootstrap': False, 'ccp_alpha': 0.0, 'class_..."
5,Statistical_Bias,0.152281,0.152943,0.093525,0.725172,0.151414,0.083923,0.723306,0.127153,0.075223,...,0.693886,0.139077,0.083592,0.735364,0.294250,0.165436,0.683618,501,RandomForestClassifier,"{'bootstrap': False, 'ccp_alpha': 0.0, 'class_..."
6,Jitter,0.041957,0.041546,0.029635,0.156252,0.042495,0.026152,0.180982,0.030900,0.020225,...,0.188027,0.035576,0.024415,0.155523,0.110553,0.076883,0.212329,501,RandomForestClassifier,"{'bootstrap': False, 'ccp_alpha': 0.0, 'class_..."
7,Per_Sample_Accuracy,0.894974,0.897924,0.978803,0.119009,0.891106,0.980839,0.130737,0.917767,0.985524,...,0.137113,0.908755,0.982501,0.116204,0.746808,0.942707,0.154659,501,RandomForestClassifier,"{'bootstrap': False, 'ccp_alpha': 0.0, 'class_..."
8,Label_Stability,0.938851,0.939356,0.957680,0.762883,0.938189,0.961752,0.738526,0.955026,0.971098,...,0.725775,0.948292,0.965049,0.768210,0.837345,0.885865,0.690682,501,RandomForestClassifier,"{'bootstrap': False, 'ccp_alpha': 0.0, 'class_..."
9,TPR,0.976331,0.975449,1.000000,0.000000,0.977500,1.000000,0.000000,0.985589,1.000000,...,0.000000,0.981481,1.000000,0.000000,0.908397,1.000000,0.000000,501,RandomForestClassifier,"{'bootstrap': False, 'ccp_alpha': 0.0, 'class_..."


2023-08-06 19:06:25 experiment_interface.py INFO    : Experiment run was successful!
INFO:root:Experiment run was successful!


### Experiment iteration 6

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 6
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Law_School_alpha_0.0_20230806__154602.csv',
    'tuning_results_Law_School_alpha_0.2_20230806__161805.csv',
    'tuning_results_Law_School_alpha_0.3_20230806__203110.csv',
    'tuning_results_Law_School_alpha_0.5_20230806__195848.csv',
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_model_params_for_mult_repair_levels(experiment_seed)

In [ ]:
run_exp_iter_with_disparate_impact(data_loader=exp_iter_data_loader,
                                   experiment_seed=experiment_seed,
                                   test_set_fraction=TEST_SET_FRACTION,
                                   db_writer_func=db_writer_func,
                                   fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                   models_params_for_tuning=models_params_for_tuning,
                                   metrics_computation_config=metrics_computation_config,
                                   custom_table_fields_dct=custom_table_fields_dct,
#                                    with_tuning=True,
                                   with_tuning=False,
                                   tuned_params_df_paths=tuned_params_df_paths,
                                   save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                   verbose=True, 
                                   dataset_name=DATASET_NAME)